<a href="https://colab.research.google.com/github/K-SAHASRA/GNN-modelling/blob/main/gnn_2_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install pandas
!pip install scikit-learn
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu117.html





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 35.7 MB/s eta 0:00:00


In [ ]:

!pip uninstall torch-scatter -y
!pip uninstall torch-geometric -y
!pip uninstall torch-sparse -y
!pip install torch==2.0.0+cu117 torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cu117.html


Found existing installation: torch-scatter 2.1.2+pt20cu117
Uninstalling torch-scatter-2.1.2+pt20cu117:
  Successfully uninstalled torch-scatter-2.1.2+pt20cu117
Found existing installation: torch-geometric 2.6.0
Uninstalling torch-geometric-2.6.0:
  Successfully uninstalled torch-geometric-2.6.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 445.1 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00
INFO: pip

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
  Using cached https://data.pyg.org/whl/torch-2.0.0%2Bcu117/torch_scatter-2.1.2%2Bpt20cu117-cp310-cp310-linux_x86_64.whl (10.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 24.9 MB/s eta 0:00:00
  Using cached torch_geometric-2.6.0-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.0-py3-none-any.whl (1.1 MB)


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving 100-iterations data combined.csv to 100-iterations data combined (1).csv


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving All Exp-data-as-joinbyfield-2024-07-05 09_44_21.csv to All Exp-data-as-joinbyfield-2024-07-05 09_44_21.csv


In [ ]:
import torch
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_scatter import scatter_mean


train_file_path = '/content/100-iterations data combined.csv'
train_data = pd.read_csv(train_file_path, low_memory=False)

test_file_path = '/content/All Exp-data-as-joinbyfield-2024-07-05 09_44_21.csv'
test_data = pd.read_csv(test_file_path, low_memory=False)

def preprocess_data(data):

    #data = data.drop(columns=['sep=', 'Unnamed: 1'], errors='ignore')

    if 'Time' in data.columns:
        data['Time'] = data['Time'].astype(str)
        data_cleaned = data.drop(columns=['Time'])
    else:
        data_cleaned = data.copy()

    def custom_convert(series):
        series = series.str.replace(' K', '000')
        series = series.str.replace(' M', '000000')
        return pd.to_numeric(series, errors='coerce')

    if 'Number of Users' in data_cleaned.columns:
        data_cleaned['Number of Users'] = pd.to_numeric(data['Number of Users'], errors='coerce')
        data_cleaned = data_cleaned[data_cleaned['Number of Users'] != 0]

    columns_to_remove = [
        "Median Response Time",
        "Number of Users",
        "Current Response Time Percentile 95",
        "Current Response Time Percentile 50"
    ]
    data_cleaned = data_cleaned.drop(columns=columns_to_remove)

    data_cleaned = data_cleaned.apply(lambda col: custom_convert(col) if col.dtype == 'object' else col)
    data_cleaned = data_cleaned.fillna(0)

    if 'Average Response Time' in data_cleaned.columns:
      data_cleaned = data_cleaned[data_cleaned['Average Response Time'] != 0]


    return data_cleaned
train_data_cleaned = preprocess_data(train_data)
test_data_cleaned = preprocess_data(test_data)



In [ ]:

test_data_cleaned = preprocess_data(test_data)


ground_truth = test_data_cleaned['Average Response Time'].values[time_window:]

ground_truth_df = pd.DataFrame({
    'Time Step': range(time_window + 1, time_window + 1 + len(ground_truth)),
    'Actual Average Response Time': ground_truth
})

ground_truth_file_path = '/content/ground_truth_values.csv'
ground_truth_df.to_csv(ground_truth_file_path, index=False)

files.download(ground_truth_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:



common_columns = train_data_cleaned.columns.intersection(test_data_cleaned.columns)
train_data_cleaned = train_data_cleaned[common_columns]
test_data_cleaned = test_data_cleaned[common_columns]

scaler = MinMaxScaler()
normalized_train_data = scaler.fit_transform(train_data_cleaned)
normalized_test_data = scaler.transform(test_data_cleaned)

target_scaler = MinMaxScaler()
target_column_train = train_data_cleaned[['Average Response Time']]
target_scaler.fit(target_column_train)


normalized_train_data = torch.tensor(normalized_train_data, dtype=torch.float32)
normalized_test_data = torch.tensor(normalized_test_data, dtype=torch.float32)

def sliding_window_view(data, window_size):
    windows = []
    for i in range(len(data) - window_size):
        windows.append(data[i:i + window_size])
    return torch.stack(windows)


time_window = 10
train_node_features_time_series = sliding_window_view(normalized_train_data, time_window)
test_node_features_time_series = sliding_window_view(normalized_test_data, time_window)

average_response_time_column = 24
train_target_time_series = torch.tensor(normalized_train_data[time_window:, average_response_time_column], dtype=torch.float32).view(-1, 1)

num_nodes = train_node_features_time_series.size(1)
edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()

batch_size = train_node_features_time_series.size(0)

batch_index = torch.arange(batch_size).repeat_interleave(num_nodes)

train_graph_data = Data(x=train_node_features_time_series.view(-1, train_node_features_time_series.size(2)),
                        edge_index=edge_index, y=train_target_time_series, batch=batch_index)

test_graph_data = Data(x=test_node_features_time_series.view(-1, test_node_features_time_series.size(2)),
                       edge_index=edge_index, batch=batch_index)





# # Create Data objects for training and test sets
# train_graph_data = Data(x=train_node_features_time_series.view(-1, train_node_features_time_series.size(2)),
#                         edge_index=edge_index, y=train_target_time_series)

# test_graph_data = Data(x=test_node_features_time_series.view(-1, test_node_features_time_series.size(2)),
#                        edge_index=edge_index)



<ipython-input-41-227284b4e681>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_target_time_series = torch.tensor(normalized_train_data[time_window:, average_response_time_column], dtype=torch.float32).view(-1, 1)


In [ ]:
# class GNNModel(nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GNNModel, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
#         self.fc1 = nn.Linear(hidden_channels * 4, hidden_channels)
#         self.fc2 = nn.Linear(hidden_channels, out_channels)
#         self.output_activation = nn.ReLU()

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index
#         x = self.conv1(x, edge_index)
#         x = self.output_activation(x)
#         x = self.att1(x, edge_index)
#         x = torch.relu(x)
#         x = self.fc1(x)
#         x = torch.relu(x)
#         x = self.fc2(x)
#         x = self.output_activation(x)

#         # Removed scatter_mean to avoid issues with batch indices
#         return x.mean(dim=0)  # Simple average over all nodes


# # Model instantiation
# model = GNNModel(in_channels=train_node_features_time_series.size(2), hidden_channels=32, out_channels=1)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.MSELoss()

# # Training the model
# def train(model, data, epochs=100):
#     model.train()
#     for epoch in range(epochs):
#         optimizer.zero_grad()
#         out = model(data)
#         loss = criterion(out, data.y)
#         loss.backward()
#         optimizer.step()
#         print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# # Train the model
# train(model, train_graph_data)

# # Ensure shapes match expected sizes
# print(f"Training Data Shape: {train_graph_data.x.shape}")
# print(f"Test Data Shape: {test_graph_data.x.shape}")
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.att1 = GATConv(hidden_channels, hidden_channels, heads=4, concat=True)
        self.fc1 = nn.Linear(hidden_channels * 4, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, out_channels)
        self.output_activation = nn.ReLU()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.output_activation(x)
        x = self.att1(x, edge_index)
        x = torch.relu(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x


def train(model, data, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)

        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

train(model, train_graph_data)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([11586, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.014648144133388996
Epoch 2, Loss: 0.014648144133388996
Epoch 3, Loss: 0.014648123644292355
Epoch 4, Loss: 0.014648094773292542
Epoch 5, Loss: 0.014648071490228176
Epoch 6, Loss: 0.014648061245679855
Epoch 7, Loss: 0.014648063108325005
Epoch 8, Loss: 0.0146480742841959
Epoch 9, Loss: 0.014648087322711945
Epoch 10, Loss: 0.014648094773292542
Epoch 11, Loss: 0.014648092910647392
Epoch 12, Loss: 0.014648083597421646
Epoch 13, Loss: 0.014648071490228176
Epoch 14, Loss: 0.014648063108325005
Epoch 15, Loss: 0.014648061245679855
Epoch 16, Loss: 0.01464806217700243
Epoch 17, Loss: 0.014648065902292728
Epoch 18, Loss: 0.014648071490228176
Epoch 19, Loss: 0.0146480742841959
Epoch 20, Loss: 0.014648071490228176
Epoch 21, Loss: 0.014648069627583027
Epoch 22, Loss: 0.014648063108325005
Epoch 23, Loss: 0.014648061245679855
Epoch 24, Loss: 0.014648059383034706
Epoch 25, Loss: 0.014648061245679855
Epoch 26, Loss: 0.01464806217700243
Epoch 27, Loss: 0.014648063108325005
Epoch 28, Loss: 

In [ ]:

test_graphs = []
for i in range(test_node_features_time_series.size(0)):
    test_graph = Data(x=test_node_features_time_series[i].view(-1, test_node_features_time_series.size(2)),
                      edge_index=edge_index)
    test_graphs.append(test_graph)



In [ ]:
def predict_individual_graphs(model, test_graphs):
    model.eval()
    predictions = []

    with torch.no_grad():
        for graph in test_graphs:
            pred = model(graph)
            predictions.append(pred)


    return torch.stack(predictions, dim=0)

predicted_response_times = predict_individual_graphs(model, test_graphs)
predicted_response_times_original_scale = target_scaler.inverse_transform(predicted_response_times.numpy())

for i, pred in enumerate(predicted_response_times_original_scale, start=time_window + 1):
    print(f"Time step {i}: Predicted Average Response Time = {pred[0]:.4f}")


Time step 11: Predicted Average Response Time = 42.0411
Time step 12: Predicted Average Response Time = 41.6388
Time step 13: Predicted Average Response Time = 40.9026
Time step 14: Predicted Average Response Time = 40.6035
Time step 15: Predicted Average Response Time = 41.0604
Time step 16: Predicted Average Response Time = 40.8190
Time step 17: Predicted Average Response Time = 41.8024
Time step 18: Predicted Average Response Time = 40.9721
Time step 19: Predicted Average Response Time = 41.5423
Time step 20: Predicted Average Response Time = 41.8564
Time step 21: Predicted Average Response Time = 40.4121
Time step 22: Predicted Average Response Time = 38.3245
Time step 23: Predicted Average Response Time = 36.8299
Time step 24: Predicted Average Response Time = 35.1574
Time step 25: Predicted Average Response Time = 33.7250
Time step 26: Predicted Average Response Time = 32.9431
Time step 27: Predicted Average Response Time = 31.2846
Time step 28: Predicted Average Response Time = 

In [ ]:

predicted_values = predicted_response_times_original_scale

time_window = 10
time_steps = list(range(time_window + 1, time_window + 1 + len(predicted_values)))

predicted_values_flat = [pred[0] for pred in predicted_values]

predictions_df = pd.DataFrame({
    'Time Step': time_steps,
    'Predicted Average Response Time': predicted_values_flat
})

output_file_path = '/content/predicted_response_times_with_timestamps.csv'
predictions_df.to_csv(output_file_path, index=False)

files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Function to modify the 'Number of Users' column and predict response times
def predict_for_different_user_counts(model, test_data_cleaned, user_values, scaler, target_scaler, time_window, edge_index):
    results = {}

    for users in user_values:
        # Make a copy of the test dataset
        modified_test_data = test_data_cleaned.copy()

        # Set the 'Number of Users' column to the specified value
        if 'Number of Users' in modified_test_data.columns:
            modified_test_data['Number of Users'] = 100

        # Normalize the modified test data
        normalized_modified_test_data = scaler.transform(modified_test_data)
        normalized_modified_test_data = torch.tensor(normalized_modified_test_data, dtype=torch.float32)

        # Create sliding windows for the modified test data
        test_node_features_time_series = sliding_window_view(normalized_modified_test_data, time_window)

        # Create test graphs for the modified test data
        test_graphs = []
        for i in range(test_node_features_time_series.size(0)):
            test_graph = Data(x=test_node_features_time_series[i].view(-1, test_node_features_time_series.size(2)),
                              edge_index=edge_index)
            test_graphs.append(test_graph)

        # Predict using the model
        predicted_response_times = predict_individual_graphs(model, test_graphs)

        # Inverse transform to get predictions in the original scale
        predicted_response_times_original_scale = target_scaler.inverse_transform(predicted_response_times.numpy())

        # Store the predicted values for this user count
        results[users] = predicted_response_times_original_scale

    return results

# Define different 'Number of Users' values for prediction (e.g., low, medium, high)
user_values = [10, 100, 1000, 5000]  # You can modify these as per your data

# Get predictions for each variation of 'Number of Users'
predicted_results = predict_for_different_user_counts(
    model,
    test_data_cleaned,
    user_values,
    scaler,
    target_scaler,
    time_window,
    edge_index
)

# Print the results
for users, predictions in predicted_results.items():
    print(f"Predictions for {users} users:")
    for i, pred in enumerate(predictions, start=time_window + 1):
        print(f"Time step {i}: Predicted Average Response Time = {pred[0]:.4f}")


NameError: name 'model' is not defined